In [75]:
from operator import mul
from scipy.special import binom
from functools import reduce
from itertools import product
from math import factorial

In [12]:
def partitions(n):
    L = [[1]]
    for _ in range(n-1):
        L2 = []
        for l in L:
            L2.append(l + [1])
            L2.append(l[:-1] + [l[-1] + 1])
        L = L2
    return L

In [87]:
def fixed_length_partitions(n, num_partitions):
    assert n >= num_partitions
    L = [[]]
    for i in range(num_partitions - 1):
        L2 = []
        for l in L:
            L2 += [l + [k] for k in range(1, n - sum(l) + 1 - (num_partitions - 1 - i))]
        L = L2
    for l in L:
        l.append(n - sum(l))
    #print(L)
    return L

In [88]:
def count_programs(arities, predicates_with_arity, num_variables, num_constants, max_num_nodes, max_num_clauses):
    num_predicates = sum(predicates_with_arity)
    def arity(a):
        'The arity of predicate indexed at a'
        i = 0
        while (a + 1 > predicates_with_arity[i]):
            a -= predicates_with_arity[i]
            i += 1
        return arities[i]
    
    A = sum([p_a*(num_variables+num_constants)**a for (a, p_a) in zip(arities, predicates_with_arity)])
    #print('atoms:', A)

    def T(n):
        if n == 1:
            return A
        return T(n-1) + 2 * sum([reduce(mul, [T(k) for k in partition], 1) for partition in partitions(n-1)])
    C = 1 + sum([T(n) for n in range(1, max_num_nodes + 1)])
    #print('bodies:', C)

    def H(a):
        return int(sum(binom(a, v) * num_constants**(a-v) *
                       sum(binom(v-1, k)for k in range(num_variables)) for v in range(1, a + 1)))

    s = 0
    for n in range(num_predicates, max_num_clauses + 1):
        for partition in fixed_length_partitions(n, num_predicates):
            m = int(reduce(mul, [binom(C, h) * H(arity(i))**h for i,h in enumerate(partition)], 1))
            #print(m)
            s += m
    return s

In [95]:
arities = [[1], [2], [1, 1], [3], [2, 1], [1, 1, 1], [4], [3, 1], [2, 2], [2, 1, 1], [1, 1, 1, 1]]
r = list(range(1, 5))
predicates_with_arity = {1: r, 2: list(product(r, r)), 3: list(product(r, r, r)), 4: list(product(r, r, r, r))}
num_variables = r
num_constants = range(0, 4)
max_num_nodes = r
MAX = 100000

f = open('program_counts.csv', 'w+')
for arity in arities:
    for pred in predicates_with_arity[len(arity)]:
        if isinstance(pred, tuple):
            pred = list(pred)
        elif not isinstance(pred, list):
            pred = [pred]
        num_pred = sum(pred)
        for num_var in num_variables:
            for num_const in num_constants:
                for max_nodes in max_num_nodes:
                    for max_clauses in range(num_pred, num_pred + 6):
                        count = count_programs(arity, pred, num_var, num_const, max_nodes, max_clauses)
                        if count > MAX:
                            break
                        d = [arity, pred, num_var, num_const, max_nodes, max_clauses, count]
                        s = ';'.join([str(t) for t in d])
                        f.write(s+'\n')
f.close()

In [89]:
count_programs([1], [1], 1, 0, 1, 2)

3

In [73]:
fixed_length_partitions(5, 3)

[[1, 1, 3], [1, 2, 2], [1, 3, 1], [2, 1, 2], [2, 2, 1], [3, 1, 1]]


[[1, 1, 3], [1, 2, 2], [1, 3, 1], [2, 1, 2], [2, 2, 1], [3, 1, 1]]